In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import os
import time
import matplotlib.pyplot as plt

# Set constants
BATCH_SIZE = 32
IMAGE_SIZE = (128, 128)
NOISE_DIM = 100
EPOCHS = 1000
SAVE_DIR = '/kaggle/working/'

# Create directory to save generated images
os.makedirs(SAVE_DIR, exist_ok=True)


2025-04-14 15:52:03.589262: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744645923.784323      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744645923.838823      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# Data loading and augmentation
def preprocess_image(image):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image, max_delta=0.1)
    image = tf.image.random_contrast(image, lower=0.9, upper=1.1)
    image = tf.image.random_saturation(image, lower=0.9, upper=1.1)
    image = tf.image.random_hue(image, max_delta=0.05)
    image = tf.clip_by_value(image, 0.0, 255.0)
    image = (image - 127.5) / 127.5  # Normalize to [-1, 1]
    return image

dataset = tf.keras.utils.image_dataset_from_directory(
    "/kaggle/input/breakhis/BreaKHis_v1/BreaKHis_v1/histology_slides/breast/benign/SOB/phyllodes_tumor",
    labels=None,
    label_mode=None,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True
)

dataset = dataset.map(lambda x: tf.map_fn(preprocess_image, x))
dataset = dataset.prefetch(tf.data.AUTOTUNE)


Found 453 files.


I0000 00:00:1744645937.032851      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [3]:
# Weight initializer
weight_init = tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.02)

# Generator model
def make_generator_model():
    model = tf.keras.Sequential([
        layers.Dense(8*8*512, use_bias=False, input_shape=(NOISE_DIM,), kernel_initializer=weight_init),
        layers.BatchNormalization(),
        layers.LeakyReLU(alpha=0.2),

        layers.Reshape((8, 8, 512)),

        layers.Conv2DTranspose(256, 5, strides=2, padding='same', use_bias=False, kernel_initializer=weight_init),
        layers.BatchNormalization(),
        layers.LeakyReLU(alpha=0.2),

        layers.Conv2DTranspose(128, 5, strides=2, padding='same', use_bias=False, kernel_initializer=weight_init),
        layers.BatchNormalization(),
        layers.LeakyReLU(alpha=0.2),

        layers.Conv2DTranspose(64, 5, strides=2, padding='same', use_bias=False, kernel_initializer=weight_init),
        layers.BatchNormalization(),
        layers.LeakyReLU(alpha=0.2),

        layers.Conv2DTranspose(3, 5, strides=2, padding='same', use_bias=False, activation='tanh', kernel_initializer=weight_init)
    ])
    return model


In [4]:
# Discriminator model
def make_discriminator_model():
    model = tf.keras.Sequential([
        layers.Conv2D(64, 5, strides=2, padding='same', input_shape=[128, 128, 3], kernel_initializer=weight_init),
        layers.LeakyReLU(alpha=0.2),
        layers.Dropout(0.2),

        layers.Conv2D(128, 5, strides=2, padding='same', kernel_initializer=weight_init),
        layers.LeakyReLU(alpha=0.2),
        layers.Dropout(0.2),

        layers.Conv2D(256, 5, strides=2, padding='same', kernel_initializer=weight_init),
        layers.LeakyReLU(alpha=0.2),
        layers.Dropout(0.2),

        layers.Conv2D(512, 5, strides=2, padding='same', kernel_initializer=weight_init),
        layers.LeakyReLU(alpha=0.2),
        layers.Dropout(0.2),

        layers.Flatten(),
        layers.Dense(1)
    ])
    return model


In [5]:
# Loss functions
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output)*0.9, real_output)  # Label smoothing
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    return real_loss + fake_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

# Instantiate models and optimizers
generator = make_generator_model()
discriminator = make_discriminator_model()

generator_optimizer = tf.keras.optimizers.Adam(1e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4, beta_1=0.5)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
# Training step
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, NOISE_DIM])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss


In [7]:
def train(dataset, epochs):
    for epoch in range(epochs):
        start = time.time()
        for image_batch in dataset:
            train_step(image_batch)
        print(f"Epoch {epoch+1}/{epochs} completed in {time.time() - start:.2f}s")


In [8]:
EPOCHS = 10000
train(dataset, EPOCHS)


E0000 00:00:1744645945.624720      31 meta_optimizer.cc:966] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_1_2/dropout_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
I0000 00:00:1744645945.976574      96 cuda_dnn.cc:529] Loaded cuDNN version 90300


Epoch 1/10000 completed in 12.78s
Epoch 2/10000 completed in 1.91s
Epoch 3/10000 completed in 1.84s
Epoch 4/10000 completed in 1.82s
Epoch 5/10000 completed in 2.18s
Epoch 6/10000 completed in 1.91s
Epoch 7/10000 completed in 1.84s
Epoch 8/10000 completed in 1.88s
Epoch 9/10000 completed in 1.83s
Epoch 10/10000 completed in 1.84s
Epoch 11/10000 completed in 1.84s
Epoch 12/10000 completed in 1.83s
Epoch 13/10000 completed in 1.89s
Epoch 14/10000 completed in 1.84s
Epoch 15/10000 completed in 1.83s
Epoch 16/10000 completed in 1.84s
Epoch 17/10000 completed in 1.82s
Epoch 18/10000 completed in 1.88s
Epoch 19/10000 completed in 1.84s
Epoch 20/10000 completed in 1.86s
Epoch 21/10000 completed in 1.83s
Epoch 22/10000 completed in 2.09s
Epoch 23/10000 completed in 2.02s
Epoch 24/10000 completed in 1.88s
Epoch 25/10000 completed in 1.83s
Epoch 26/10000 completed in 1.86s
Epoch 27/10000 completed in 1.84s
Epoch 28/10000 completed in 1.89s
Epoch 29/10000 completed in 1.91s
Epoch 30/10000 complet

In [9]:
def generate_images_to_folder(generator, num_images, save_dir):
    noise = tf.random.normal([num_images, 100])
    generated_images = generator(noise, training=False)
    generated_images = (generated_images * 127.5 + 127.5).numpy().astype(np.uint8)

    for i in range(num_images):
        img = Image.fromarray(generated_images[i])
        img.save(os.path.join(save_dir, f"gan_{i}.png"))


In [10]:
import numpy as np
from PIL import Image
import os
import tensorflow as tf

In [11]:
generate_images_to_folder(generator, num_images=750, save_dir=SAVE_DIR )


In [12]:
import os
import shutil

# Create a folder for subset images
subset_dir = '/kaggle/working/gan_subset3'
os.makedirs(subset_dir, exist_ok=True)

# Copy only first 100 images
all_images = sorted([f for f in os.listdir('/kaggle/working') if f.startswith('gan_') and f.endswith('.png')])
for img in all_images[:750]:
    shutil.copy(f"/kaggle/working/{img}", f"{subset_dir}/{img}")

# Zip the subset
shutil.make_archive("/kaggle/working/gan_subset3", 'zip', subset_dir)


'/kaggle/working/gan_subset3.zip'